In [44]:
import pandas as pd
from gensim.models import Word2Vec
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import matplotlib.pyplot as plt
import re
import string
import nltk
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import pickle


In [45]:
import warnings
warnings.filterwarnings("ignore")                     
import pandas as pd
import os

# Step 1: Load the corpus
content = []
file_names = []
path = '/mnt/c/Users/alamm9/Desktop/text_file_train_test/train_set'
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r') as file:
        content.append(file.read())
        file_names.append(filename)

# Step 2: Create a dataframe from the corpus
df_train = pd.DataFrame({'Filename': file_names, 'Content': content})


In [3]:
content = []
file_names = []
path = '/mnt/c/Users/alamm9/Desktop/text_file_train_test/In_sample_test'
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r') as file:
        content.append(file.read())
        file_names.append(filename)

# Step 2: Create a dataframe from the corpus
df_test = pd.DataFrame({'Filename': file_names, 'Content': content})


In [ ]:
content = []
file_names = []
path = '/mnt/c/Users/alamm9/Desktop/text_file_train_test/Out_of_sample_test'
for filename in os.listdir(path):
    with open(os.path.join(path, filename), 'r') as file:
        content.append(file.read())
        file_names.append(filename)

# Step 2: Create a dataframe from the corpus
df_test_out = pd.DataFrame({'Filename': file_names, 'Content': content})


In [25]:
df_foreign=create_dataframe_from_files("/mnt/c/Users/alamm9/Desktop/text_file_train_test/fake.csv")

NotADirectoryError: [Errno 20] Not a directory: '/mnt/c/Users/alamm9/Desktop/text_file_train_test/fake.csv'

In [ ]:
df_foreign = pd.read_csv("/mnt/c/Users/alamm9/Desktop/text_file_train_test/fake.csv")

In [46]:
def preprocess_text(df, column):
    


    # Remove special characters and numbers
    df[column] = df[column].apply(lambda text: re.sub(r'[^a-zA-Z\s]', '', str(text)))
    
    # Convert to lowercase
    df[column] = df[column].str.lower()
    
    # Remove multiple occurrences of 'i' characters
    df[column] = df[column].apply(lambda text: re.sub(r'(i{2,})', 'i', str(text)))
    
    # Remove single alphabets
    df[column] = df[column].apply(lambda text: re.sub(r'\b[a-zA-Z]\b', '', str(text)))
    
    # Tokenize the text
    df[column] = df[column].apply(lambda text: nltk.word_tokenize(text))
    
    # Remove stopwords
    stop_words = set(stopwords.words('english'))
    df[column] = df[column].apply(lambda tokens: [token for token in tokens if token not in stop_words])
    
    # Lemmatize the tokens
    lemmatizer = WordNetLemmatizer()
    df[column] = df[column].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])
    
    # Remove 'mmddyyy'
    df[column] = df[column].apply(lambda text: re.sub(r'(\b[mmddyyy]+\b)', '', str(text)))
    
    # Remove extra whitespace
    df[column] = df[column].apply(lambda text: re.sub(r'\s+', '', str(text)))
    
    # Join the tokens back into a string
    df[column] = df[column].apply(lambda tokens: ''.join(tokens))
    
    return df

In [47]:

df_train = preprocess_text(df_train, "Content")
#df_test = preprocess_text(df_test, "Content")
#df_foreign = preprocess_text(df_foreign, "text")
#df_test_out = preprocess_text(df_test_out, "Content")

In [ ]:
#df_train["Content"]

In [48]:
from sklearn.model_selection import train_test_split

# Splitting the dataset
df_train, df_validation = train_test_split(df_train, test_size=0.2, random_state=40)

In [6]:
from termcolor import colored


In [57]:
import gensim
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from numpy import percentile
import os  

import joblib
import h5py



def w2v(df_train,df, tfidf=bool):# , percentile_value
    
    
    if (tfidf==0):
        # Train Word2Vec model
        model = gensim.models.Word2Vec(df_train["Content"], vector_size=100, window=5, min_count=1, workers=4)

        # Obtain document embeddings for the training set
        train_embeddings = [np.mean([model.wv[token] for token in doc_tokens if token in model.wv], axis=0) for doc_tokens in df_train["Content"]]

        # Transform the test set embeddings using the learned transformation from the training set
        test_embeddings = [np.mean([model.wv[token] for token in doc_tokens if token in model.wv], axis=0) for doc_tokens in df["Content"]]

        # Calculate cosine similarity matrix
        similarity_matrix = cosine_similarity(train_embeddings, test_embeddings) #trian 80% and validation 20%


        # Calculate mean similarity for each row
        mean_similarity = np.mean(similarity_matrix, axis=0)
        threshold = np.percentile(mean_similarity, [10])
    
    else:
        # Initialize the TF-IDF vectorizer
        with open("vectorizer.pkl", "rb") as f:
            vectorizer = pickle.load(f)

        # Apply the same vectorizer on the test documents
        test = vectorizer.transform(df_unseen["Content"])
        

        with open("train(tfidf).pkl", "rb") as f:
            train = pickle.load(f)

        with open("threshold(tfidf).pkl", "rb") as f:
            threshold = pickle.load(f)

        similarity_matrix = cosine_similarity(train, test) #trian 80% and validation 20%
        print("similarity_matrix",similarity_matrix.shape)

        # Calculate mean similarity for each row
        mean_similarity = np.mean(similarity_matrix, axis=0)
        threshold_tfidf = np.percentile(mean_similarity, 10)
        #print("threshold(tfidf):", threshold_tfidf)

    
    
#   # Determine data drift and print document information
    messages = []
    for doc_idx, similarity_score in enumerate(mean_similarity):
        is_drift = similarity_score <= threshold
        drift_status = "𝗗𝗮𝘁𝗮 𝗗𝗿𝗶𝗳𝘁" if is_drift else "No Data Drift"
        similarity_str = "{:}".format(similarity_score)  
        message = f"Document {doc_idx + 1:<4}: Mean Similarity = {similarity_str:<20} {drift_status}"
        messages.append(message)

    # Combine all messages into a single string
    result = (messages)

    return result



In [58]:
w2v(df_train,df_test)

NameError: name 'df_unseen' is not defined

In [ ]:
# # Plot the heatmap
# plt.figure(figsize=(8, 6))
# heatmap = plt.imshow(similarity_matrix, cmap='hot', interpolation='nearest')
# plt.colorbar(heatmap)
# plt.title('Cosine Similarity Heatmap')
# plt.xlabel('Training Set')
# plt.ylabel('Test Set')
# plt.show()

In [ ]:
# # Plot the histogram
# plt.figure(figsize=(8, 6))
# plt.hist(cosine_similarity_values, bins=10, color='skyblue')
# plt.xlabel('Cosine Similarity')
# plt.ylabel('Frequency')
# plt.title('Histogram of Cosine Similarity')
# plt.show()

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

import pickle
# def tfidf(train, column_train):
    
#     # Initialize the TF-IDF vectorizer
#     vectorizer = TfidfVectorizer()
    
#     # Fit the vectorizer on the training documents
#     train = vectorizer.fit_transform(train[column_train])
    
#     # Save the train variable to a file
#     with open('train_tfidf.pkl', 'wb') as file:
#         pickle.dump(train, file)


In [ ]:
# tfidf(df_train, "Content")

In [49]:
from sklearn.feature_extraction.text import TfidfVectorizer


In [56]:
 # Initialize the TF-IDF vectorizer
vectorizer = TfidfVectorizer()
train = vectorizer.fit_transform(df_train["Content"])

# with open("train(tfidf).pkl", "wb") as f:
#     pickle.dump(train, f)
    
    print("train(tfidf):", train)

    
# with open("vectorizer.pkl", "wb") as f:
#     pickle.dump(vectorizer, f)
    
    print("vectorizer(tfidf):", vectorizer)


# Apply the same vectorizer on the test documents
test = vectorizer.transform(df_validation["Content"])

similarity_matrix1 = cosine_similarity(train, test) #trian 80% and validation 20%
print("similarity_matrix",similarity_matrix1.shape)

# Calculate mean similarity for each row
mean_similarity1 = np.mean(similarity_matrix1, axis=0)
# print(mean_similarity1)
# print(mean_similarity1.shape)
threshold_tfidf = np.percentile(mean_similarity1, 10)
print("threshold(tfidf):", threshold_tfidf)
# with open("threshold(tfidf).pkl", "wb") as f:
#      pickle.dump(threshold_tfidf, f)

train(tfidf):   (0, 17822)	0.04779348001825221
  (0, 13367)	0.023050959410843275
  (0, 7940)	0.019033249489315292
  (0, 13705)	0.019398573642811022
  (0, 8237)	0.04077177354645408
  (0, 2131)	0.013993877358571556
  (0, 11598)	0.03450286941654411
  (0, 12676)	0.026162377526183916
  (0, 4524)	0.02015607729617889
  (0, 6922)	0.04779348001825221
  (0, 4320)	0.04779348001825221
  (0, 4070)	0.03251024424045488
  (0, 10599)	0.03139977890194963
  (0, 964)	0.03276482338373805
  (0, 2650)	0.03115856413676053
  (0, 965)	0.02661108331853525
  (0, 2655)	0.02949991580409821
  (0, 3797)	0.03092612278773077
  (0, 14915)	0.02627163544600588
  (0, 960)	0.03246749383607051
  (0, 3694)	0.029685187694248976
  (0, 3059)	0.022601125287185542
  (0, 12524)	0.01596301988344955
  (0, 8783)	0.03582998243585233
  (0, 5649)	0.08849974741229462
  :	:
  (1299, 9743)	0.022674467092546315
  (1299, 5335)	0.05149278225145697
  (1299, 16045)	0.02680062644682873
  (1299, 16590)	0.015921156045539637
  (1299, 2328)	0.0738572